In [7]:
import os
import json
from qiskit.circuit.library import QuantumVolume, QFT
from disqco.circuits.cp_fraction import cp_fraction
from disqco.circuits.QAOA import QAOA_random
from QASMBench.interface.qiskit import QASMBenchmark
from qiskit import transpile, QuantumCircuit

# path to the root directory of QASMBench
path = "QASMBench"
category = "large" 
num_qubits_list = list(range(3, 100))
remove_final_measurements = True
do_transpile = True  
basis_gates = ['u', 'cp']
transpile_args = { 'basis_gates': basis_gates}
bm = QASMBenchmark(path, category, num_qubits_list=num_qubits_list, do_transpile=do_transpile, **transpile_args, remove_final_measurements=remove_final_measurements)

# 1. Specify the folder containing your JSON files
folder_path = "/Users/ftb123/MLQCP_FM/benchmarking/JSON_data/Used"

# 2. Get a list of all JSON files in the folder
file_list = [f for f in os.listdir(folder_path) if f.endswith(".json")]

# (Optional) Sort the list of files if you want them processed in alphabetical or numerical order
file_list.sort()

# 3. Iterate through each JSON file in the folder

def write_fractions(circuit : QuantumCircuit, entry : dict):
    circuit = transpile(circuit, basis_gates=basis_gates)
    num_two_qubit_gates = circuit.count_ops()['cp']
    cost = entry['r_cost']
    time = entry['time_r']
    cost_fraction = cost/num_two_qubit_gates
    time_fraction = time/num_two_qubit_gates
    entry['num_two_qubit_gates'] = num_two_qubit_gates
    entry['cost_fraction'] = cost_fraction
    entry['time_fraction'] = time_fraction
    


In [ ]:
for filename in file_list:
    file_path = os.path.join(folder_path, filename)
    
    # 4. Open and load the JSON data
    with open(file_path, 'r') as f:
        data = json.load(f)
    
    circuit_type = filename[25:-5]
    
    if circuit_type[0:2] == "CP":
        for entry in data:
            num_qubits = entry['num_qubits']
            if 'fraction' in entry:
                fraction = entry['fraction']
            else:
                fraction = 0.5
            circuit = cp_fraction(num_qubits, num_qubits, fraction)
            write_fractions(circuit, entry)
    elif circuit_type[0:4] == "QAOA":
        for entry in data:
            num_qubits = entry['num_qubits']
            circuit = QAOA_random(num_qubits, prob=0.5, reps=1)
            write_fractions(circuit, entry)

    elif circuit_type[0:4] == "QASM":
        for entry in data:
            circuit_name = entry['circuit_name']
            circuit = bm.get(circuit_name)
            write_fractions(circuit, entry)
    elif circuit_type[0:3] == "QFT":
        for entry in data:
            num_qubits = entry['num_qubits']
            circuit = QFT(num_qubits, do_swaps=False)
            write_fractions(circuit, entry)
    elif circuit_type[0:2] == "QV":
        for entry in data:
            num_qubits = entry['num_qubits']
            circuit = QuantumVolume(num_qubits)
            write_fractions(circuit, entry)

    new_name = filename + "_fraction.json"
    new_path = os.path.join(folder_path + "/fractions/", new_name)
    
    with open(new_path, 'w') as f:
        json.dump(data, f, indent=2)


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x10368ef90>>
Traceback (most recent call last):
  File "/Users/ftb123/MLQCP_FM/.venv/lib/python3.13/site-packages/ipykernel/ipkernel.py", line 794, in _clean_thread_parent_frames
    for identity in list(thread_to_parent_header.keys()):
KeyboardInterrupt: 


In [ ]:
counter = 1
total_cost_fraction = 0
total_time_fraction = 0

fractions = {}

for filename in file_list:
    filename = filename + "_fraction.json"

    file_path = os.path.join(folder_path + "/fractions/", filename)
    
    # 4. Open and load the JSON data
    with open(file_path, 'r') as f:
        data = json.load(f)

    cost_fraction_circuit = 0
    time_fraction_circuit = 0
    inner_counter = 0

    print("Name:", filename)

    for entry in data:
        if entry['iteration'] < 5:
            cost_fraction = entry['cost_fraction']
            time_fraction = entry['time_fraction']
            cost_fraction_circuit += cost_fraction
            time_fraction_circuit += time_fraction
            inner_counter += 1
    
    avg_cost_fraction_circuit = cost_fraction_circuit/inner_counter
    avg_time_fraction_circuit = time_fraction_circuit/inner_counter

    fractions[filename] = {"cost_fraction" : avg_cost_fraction_circuit, "time_fraction" : avg_time_fraction_circuit}

    print("Cost fraction:", avg_cost_fraction_circuit)
    print("Time fraction:", avg_time_fraction_circuit)


Name: benchmark_results_MLFM-R_CP_2-12_new.json_fraction.json
Cost fraction: 0.38984791604901103
Time fraction: 0.004078376826869355
Name: benchmark_results_MLFM-R_CP_large_2-4part_new.json_fraction.json
Cost fraction: 0.3011763302542692
Time fraction: 0.0025153138052980257
Name: benchmark_results_MLFM-R_QAOA_2-4part_new.json_fraction.json
Cost fraction: 0.0448441893866055
Time fraction: 0.0063992887041751725
Name: benchmark_results_MLFM-R_QASM_large_new.json_fraction.json
Cost fraction: 0.04947923506010539
Time fraction: 0.013325105015018542
Name: benchmark_results_MLFM-R_QASM_larger_new.json_fraction.json
Cost fraction: 0.021957435291320382
Time fraction: 0.03302235241073638
Name: benchmark_results_MLFM-R_QFT_2-4part_new.json_fraction.json
Cost fraction: 0.04360141710935251
Time fraction: 0.003569727763855581
Name: benchmark_results_MLFM-R_QV_2-4part_new.json_fraction.json
Cost fraction: 0.18770323269177733
Time fraction: 0.0022197772922284338


In [ ]:
cumulative_cost_fraction = 0
cumulative_time_fraction = 0
counter = 0

with open("e-bit_fractions_MLFM-R.dat", 'w') as f:
    f.write("Circuit name, Cost fraction, Time fraction\n")

    for circuit in fractions:
        f.write(f"{circuit}, {fractions[circuit]['cost_fraction']}, {fractions[circuit]['time_fraction']}\n")
        data = fractions[circuit]
        cumulative_cost_fraction += data['cost_fraction']
        cumulative_time_fraction += data['time_fraction']
        counter += 1
        print("Name:", circuit[:30])
        print("Cost fraction:", data['cost_fraction'])
        print("Time fraction:", data['time_fraction'])


    avg_cost_fraction = cumulative_cost_fraction/counter
    avg_time_fraction = cumulative_time_fraction/counter
    f.write(f"Average: {avg_cost_fraction}, {avg_time_fraction}\n")

print("Average cost fraction:", avg_cost_fraction)
print("Average time fraction:", avg_time_fraction)

Name: benchmark_results_MLFM-R_CP_2-
Cost fraction: 0.38984791604901103
Time fraction: 0.004078376826869355
Name: benchmark_results_MLFM-R_CP_la
Cost fraction: 0.3011763302542692
Time fraction: 0.0025153138052980257
Name: benchmark_results_MLFM-R_QAOA_
Cost fraction: 0.0448441893866055
Time fraction: 0.0063992887041751725
Name: benchmark_results_MLFM-R_QASM_
Cost fraction: 0.04947923506010539
Time fraction: 0.013325105015018542
Name: benchmark_results_MLFM-R_QASM_
Cost fraction: 0.021957435291320382
Time fraction: 0.03302235241073638
Name: benchmark_results_MLFM-R_QFT_2
Cost fraction: 0.04360141710935251
Time fraction: 0.003569727763855581
Name: benchmark_results_MLFM-R_QV_2-
Cost fraction: 0.18770323269177733
Time fraction: 0.0022197772922284338
Average cost fraction: 0.14837282226320592
Average time fraction: 0.009304277402597356
